# Subject Mapping Extractions for Springer Nature

This notebook allows to select an FOR code (subject area) and a time span to produce CSV reports for the following areas: Data Competitors, Data Growth, Data Top Target Institutions and Data Top Countries. The reports are based on the [spreadsheet](https://drive.google.com/open?id=1La_GWhzw1ReFL3biYLIL_K7qMUWxXX_B) used by the SN team programmatically, as documented in the [Data Request Specs](https://docs.google.com/document/d/1haAj3SpraxzNLLOcF7_TBfrQ7chFWyRp70SueFeCPDg/edit). 



## 1. Log into the Dimensions API

This step needs to be run each time the notebooks gets re-loaded, as it sets up the API connection with the Dimensions back end. 

In [1]:
username = ""  #@param {type: "string"}
password = ""  #@param {type: "string"}
endpoint = "https://sn-insights.dimensions.ai"  #@param {type: "string"}

#
#
#
!pip install dimcli -U --quiet 
import dimcli
import pandas as pd
import time
import os
from datetime import datetime
dimcli.login(username, password, endpoint)
dsl = dimcli.Dsl()
#
#
#

You should consider upgrading via the '/Users/michele.pasin/Envs/dslqa/bin/python -m pip install --upgrade pip' command.


Dimcli - Dimensions API Client (v0.7.3)


Connected to: https://app.dimensions.ai - DSL v1.26


Method: dsl.ini file


## 2. Select Subjects and Extract Data 

Pick a year range and one or more subjects from the [FOR categories](https://sn-insights.dimensions.ai/browse/publication/for?and_facet_open_access_status_analytics). 

If you want only one subject, leave the second one to 'None'. 

If two subjects are selected, the value in the 'connector' field determine the query semantics: OR = any of the two subjects should match results; AND = both of the subjects should match the results. 

In [2]:

###################
#
# 1. SUBJECTS SELECTIONS AND SETTINGS 
#
###################



# tip 
# the subjects dropdown can be generated via 
# `str(["%s" % s for s in  sorted(dimcli.G.categories('category_for')) if len(s.split()[0]) > 2])`
#

start_year = 2015  #@param {type: "slider", min: 1980, max: 2020}
end_year = 2019  #@param {type: "slider", min: 1980, max: 2020}
subject1 = "0911 Maritime Engineering"  #@param ['0101 Pure Mathematics', '0102 Applied Mathematics', '0103 Numerical and Computational Mathematics', '0104 Statistics', '0105 Mathematical Physics', '0201 Astronomical and Space Sciences', '0202 Atomic, Molecular, Nuclear, Particle and Plasma Physics', '0203 Classical Physics', '0204 Condensed Matter Physics', '0205 Optical Physics', '0206 Quantum Physics', '0299 Other Physical Sciences', '0301 Analytical Chemistry', '0302 Inorganic Chemistry', '0303 Macromolecular and Materials Chemistry', '0304 Medicinal and Biomolecular Chemistry', '0305 Organic Chemistry', '0306 Physical Chemistry (incl. Structural)', '0307 Theoretical and Computational Chemistry', '0399 Other Chemical Sciences', '0401 Atmospheric Sciences', '0402 Geochemistry', '0403 Geology', '0404 Geophysics', '0405 Oceanography', '0406 Physical Geography and Environmental Geoscience', '0499 Other Earth Sciences', '0501 Ecological Applications', '0502 Environmental Science and Management', '0503 Soil Sciences', '0599 Other Environmental Sciences', '0601 Biochemistry and Cell Biology', '0602 Ecology', '0603 Evolutionary Biology', '0604 Genetics', '0605 Microbiology', '0606 Physiology', '0607 Plant Biology', '0608 Zoology', '0699 Other Biological Sciences', '0701 Agriculture, Land and Farm Management', '0702 Animal Production', '0703 Crop and Pasture Production', '0704 Fisheries Sciences', '0705 Forestry Sciences', '0706 Horticultural Production', '0707 Veterinary Sciences', '0799 Other Agricultural and Veterinary Sciences', '0801 Artificial Intelligence and Image Processing', '0802 Computation Theory and Mathematics', '0803 Computer Software', '0804 Data Format', '0805 Distributed Computing', '0806 Information Systems', '0807 Library and Information Studies', '0899 Other Information and Computing Sciences', '0901 Aerospace Engineering', '0902 Automotive Engineering', '0903 Biomedical Engineering', '0904 Chemical Engineering', '0905 Civil Engineering', '0906 Electrical and Electronic Engineering', '0907 Environmental Engineering', '0908 Food Sciences', '0909 Geomatic Engineering', '0910 Manufacturing Engineering', '0911 Maritime Engineering', '0912 Materials Engineering', '0913 Mechanical Engineering', '0914 Resources Engineering and Extractive Metallurgy', '0915 Interdisciplinary Engineering', '0999 Other Engineering', '1001 Agricultural Biotechnology', '1002 Environmental Biotechnology', '1003 Industrial Biotechnology', '1004 Medical Biotechnology', '1005 Communications Technologies', '1006 Computer Hardware', '1007 Nanotechnology', '1099 Other Technology', '1101 Medical Biochemistry and Metabolomics', '1102 Cardiorespiratory Medicine and Haematology', '1103 Clinical Sciences', '1104 Complementary and Alternative Medicine', '1105 Dentistry', '1106 Human Movement and Sports Science', '1107 Immunology', '1108 Medical Microbiology', '1109 Neurosciences', '1110 Nursing', '1111 Nutrition and Dietetics', '1112 Oncology and Carcinogenesis', '1113 Ophthalmology and Optometry', '1114 Paediatrics and Reproductive Medicine', '1115 Pharmacology and Pharmaceutical Sciences', '1116 Medical Physiology', '1117 Public Health and Health Services', '1199 Other Medical and Health Sciences', '1201 Architecture', '1202 Building', '1203 Design Practice and Management', '1205 Urban and Regional Planning', '1299 Other Built Environment and Design', '1301 Education Systems', '1302 Curriculum and Pedagogy', '1303 Specialist Studies In Education', '1399 Other Education', '1401 Economic Theory', '1402 Applied Economics', '1403 Econometrics', '1499 Other Economics', '1501 Accounting, Auditing and Accountability', '1502 Banking, Finance and Investment', '1503 Business and Management', '1504 Commercial Services', '1505 Marketing', '1506 Tourism', '1507 Transportation and Freight Services', '1601 Anthropology', '1602 Criminology', '1603 Demography', '1604 Human Geography', '1605 Policy and Administration', '1606 Political Science', '1607 Social Work', '1608 Sociology', '1699 Other Studies In Human Society', '1701 Psychology', '1702 Cognitive Sciences', '1799 Other Psychology and Cognitive Sciences', '1801 Law', '1899 Other Law and Legal Studies', '1901 Art Theory and Criticism', '1902 Film, Television and Digital Media', '1903 Journalism and Professional Writing', '1904 Performing Arts and Creative Writing', '1905 Visual Arts and Crafts', '1999 Other Studies In Creative Arts and Writing', '2001 Communication and Media Studies', '2002 Cultural Studies', '2003 Language Studies', '2004 Linguistics', '2005 Literary Studies', '2099 Other Language, Communication and Culture', '2101 Archaeology', '2102 Curatorial and Related Studies', '2103 Historical Studies', '2199 Other History and Archaeology', '2201 Applied Ethics', '2202 History and Philosophy of Specific Fields', '2203 Philosophy', '2204 Religion and Religious Studies', '2299 Other Philosophy and Religious Studies']
subject2 = "None"  #@param ['None', '0101 Pure Mathematics', '0102 Applied Mathematics', '0103 Numerical and Computational Mathematics', '0104 Statistics', '0105 Mathematical Physics', '0201 Astronomical and Space Sciences', '0202 Atomic, Molecular, Nuclear, Particle and Plasma Physics', '0203 Classical Physics', '0204 Condensed Matter Physics', '0205 Optical Physics', '0206 Quantum Physics', '0299 Other Physical Sciences', '0301 Analytical Chemistry', '0302 Inorganic Chemistry', '0303 Macromolecular and Materials Chemistry', '0304 Medicinal and Biomolecular Chemistry', '0305 Organic Chemistry', '0306 Physical Chemistry (incl. Structural)', '0307 Theoretical and Computational Chemistry', '0399 Other Chemical Sciences', '0401 Atmospheric Sciences', '0402 Geochemistry', '0403 Geology', '0404 Geophysics', '0405 Oceanography', '0406 Physical Geography and Environmental Geoscience', '0499 Other Earth Sciences', '0501 Ecological Applications', '0502 Environmental Science and Management', '0503 Soil Sciences', '0599 Other Environmental Sciences', '0601 Biochemistry and Cell Biology', '0602 Ecology', '0603 Evolutionary Biology', '0604 Genetics', '0605 Microbiology', '0606 Physiology', '0607 Plant Biology', '0608 Zoology', '0699 Other Biological Sciences', '0701 Agriculture, Land and Farm Management', '0702 Animal Production', '0703 Crop and Pasture Production', '0704 Fisheries Sciences', '0705 Forestry Sciences', '0706 Horticultural Production', '0707 Veterinary Sciences', '0799 Other Agricultural and Veterinary Sciences', '0801 Artificial Intelligence and Image Processing', '0802 Computation Theory and Mathematics', '0803 Computer Software', '0804 Data Format', '0805 Distributed Computing', '0806 Information Systems', '0807 Library and Information Studies', '0899 Other Information and Computing Sciences', '0901 Aerospace Engineering', '0902 Automotive Engineering', '0903 Biomedical Engineering', '0904 Chemical Engineering', '0905 Civil Engineering', '0906 Electrical and Electronic Engineering', '0907 Environmental Engineering', '0908 Food Sciences', '0909 Geomatic Engineering', '0910 Manufacturing Engineering', '0911 Maritime Engineering', '0912 Materials Engineering', '0913 Mechanical Engineering', '0914 Resources Engineering and Extractive Metallurgy', '0915 Interdisciplinary Engineering', '0999 Other Engineering', '1001 Agricultural Biotechnology', '1002 Environmental Biotechnology', '1003 Industrial Biotechnology', '1004 Medical Biotechnology', '1005 Communications Technologies', '1006 Computer Hardware', '1007 Nanotechnology', '1099 Other Technology', '1101 Medical Biochemistry and Metabolomics', '1102 Cardiorespiratory Medicine and Haematology', '1103 Clinical Sciences', '1104 Complementary and Alternative Medicine', '1105 Dentistry', '1106 Human Movement and Sports Science', '1107 Immunology', '1108 Medical Microbiology', '1109 Neurosciences', '1110 Nursing', '1111 Nutrition and Dietetics', '1112 Oncology and Carcinogenesis', '1113 Ophthalmology and Optometry', '1114 Paediatrics and Reproductive Medicine', '1115 Pharmacology and Pharmaceutical Sciences', '1116 Medical Physiology', '1117 Public Health and Health Services', '1199 Other Medical and Health Sciences', '1201 Architecture', '1202 Building', '1203 Design Practice and Management', '1205 Urban and Regional Planning', '1299 Other Built Environment and Design', '1301 Education Systems', '1302 Curriculum and Pedagogy', '1303 Specialist Studies In Education', '1399 Other Education', '1401 Economic Theory', '1402 Applied Economics', '1403 Econometrics', '1499 Other Economics', '1501 Accounting, Auditing and Accountability', '1502 Banking, Finance and Investment', '1503 Business and Management', '1504 Commercial Services', '1505 Marketing', '1506 Tourism', '1507 Transportation and Freight Services', '1601 Anthropology', '1602 Criminology', '1603 Demography', '1604 Human Geography', '1605 Policy and Administration', '1606 Political Science', '1607 Social Work', '1608 Sociology', '1699 Other Studies In Human Society', '1701 Psychology', '1702 Cognitive Sciences', '1799 Other Psychology and Cognitive Sciences', '1801 Law', '1899 Other Law and Legal Studies', '1901 Art Theory and Criticism', '1902 Film, Television and Digital Media', '1903 Journalism and Professional Writing', '1904 Performing Arts and Creative Writing', '1905 Visual Arts and Crafts', '1999 Other Studies In Creative Arts and Writing', '2001 Communication and Media Studies', '2002 Cultural Studies', '2003 Language Studies', '2004 Linguistics', '2005 Literary Studies', '2099 Other Language, Communication and Culture', '2101 Archaeology', '2102 Curatorial and Related Studies', '2103 Historical Studies', '2199 Other History and Archaeology', '2201 Applied Ethics', '2202 History and Philosophy of Specific Fields', '2203 Philosophy', '2204 Religion and Religious Studies', '2299 Other Philosophy and Religious Studies']
connector = "or"  #@param ['or', 'and']
#
#
#
#
nowfolder = datetime.now().strftime("%Y-%m-%d %H.%M.%S")
output_folder_name = nowfolder + "-" + subject1.replace(" ", "_")

def save_locally(header, dfdata):
  "save dataframe in local folder"
  global subject1, subject2, start_year, end_year, output_folder_name
  if not os.path.exists(output_folder_name):
    os.mkdir(output_folder_name)
  if subject2:
    subject2 = "_" + subject2.replace(" ", "_")
  else:
    subject2 = ""
  export = header + "_" + subject1.replace(" ", "_") + subject2 + "_" + str(start_year) + "_" + str(end_year) + ".csv"
  dfdata.to_csv(output_folder_name + "/" + export)
  print("\n..saved file '" + export)

#
#

if end_year < start_year :
  end_year = start_year

#
#

if subject1 == subject2:
  subject2 = None
if subject2 == "None":
  subject2 = None
if subject2: 
  cat_query = f"""(category_for.name="{subject1}" {connector} category_for.name="{subject2}" )"""
  cat_query_grants = f"""(FOR.name="{subject1}" {connector} FOR.name="{subject2}" )"""
else:
  cat_query = f"""category_for.name="{subject1}" """
  cat_query_grants = f"""FOR.name="{subject1}" """

print(f"""*****\n*****\nSelection:\nsubject1: {subject1}\nsubject2: {subject2} \
      \nconnector: {connector}\nyears: {start_year} - {end_year}\n*****\n*****\n""")



###################
#
# 2. DATA COMPETITORS 
#
###################


print("\n===\nData Competitors\n===\n...extracting data..\n")
#
#
#
#
q1 = """search publications where year in [{}:{}] and {} return publisher aggregate count, rcr_avg, fcr_gavg, altmetric_median limit 1000"""
df = dsl.query(q1.format(start_year, end_year, cat_query)).as_dataframe()
df.rename(columns={"id": "name", "count": "pubs"}, inplace=True)
#df
q2 = """search publications where altmetric > 0 and year in [{}:{}] and {} return publisher aggregate count limit 1000"""
df2 = dsl.query(q2.format(start_year, end_year, cat_query), verbose=False).as_dataframe()
df2.rename(columns={"id": "name", "count": "pubs_altmetric"}, inplace=True)
#df2
df3 = pd.merge(df, df2, how='left', on="name")
df3 = df3[['name', 'pubs', 'fcr_gavg', 'rcr_avg', 'altmetric_median', 'pubs_altmetric']]
df3 = df3.fillna(0) # fill empty values
df3['pubs_altmetric'] = df3['pubs_altmetric'].astype('int') # make col int (from float)
df3['pubs_altmetric_prc'] = ((df3['pubs_altmetric'] * 100) / df3['pubs']).round(2) # add % representation
#

save_locally("DataCompetitor", df3)
#
# df3



###################
#
# 2. DATA GROWTH 
#
###################

print("\n===\nData Growth\n===\n...extracting data..\n")
#
#
#
#
years = [x for x in range(start_year, end_year+1)]
row_labels = ['Total Publications', 'Closed', 'All OA', 'Green, Submitted','Pure Gold',  'Bronze', 'Hybrid', 'Green, Published', 'Green, Accepted' ]
df = pd.DataFrame(columns=years, index=row_labels)
#
#
#
# TIP query payload has this shape
# {'_stats': {'total_count': 13920},
#  'open_access_categories': [{'count': 8075, 'id': 'closed', 'name': 'Closed'},
#   {'count': 5845, 'id': 'oa_all', 'name': 'All OA'},
#   {'count': 2060, 'id': 'gold_bronze', 'name': 'Bronze'},
#   {'count': 2046, 'id': 'gold_pure', 'name': 'Pure Gold'},
#   {'count': 680, 'id': 'gold_hybrid', 'name': 'Hybrid'},
#   {'count': 467, 'id': 'green_acc', 'name': 'Green, Accepted'},
#   {'count': 448, 'id': 'green_pub', 'name': 'Green, Published'},
#   {'count': 144, 'id': 'green_sub', 'name': 'Green, Submitted'}]}
#
#
#
for y in years:
  q1 = """search publications where year = {} and {} return open_access_categories[name]"""
  data = dsl.query(q1.format(y, cat_query))
  df.at['Total Publications', y] = data.count_total
  #
  for l in row_labels:
    for cat in data.open_access_categories:
      if cat['name'] == l:
        df.at[l, y] = cat['count']

  time.sleep(1)
df = df.fillna(0) # fill empty values

#
save_locally("DataGrowth", df)



###################
#
# 3. DATA TOP TARGET INSTITUTIONS 
#
###################


print("\n===\nData Top Target Institutions\n===\n...extracting data..\n")
#
#
#
#
q1 = """search publications where year in [{}:{}] and {} return funders aggregate count, rcr_avg, fcr_gavg, altmetric_median limit 1000"""
df = dsl.query(q1.format(start_year, end_year, cat_query)).as_dataframe()
df.rename(columns={"country_name": "country", "count": "pubs"}, inplace=True)
df = df[['name', 'country', 'pubs', 'rcr_avg', 'fcr_gavg', 'altmetric_median', ]]
df['type'] = "funders"
# df
#
#
q2 = """search publications where year in [{}:{}] and {} return research_orgs aggregate count, rcr_avg, fcr_gavg, altmetric_median limit 1000"""
df2 = dsl.query(q2.format(start_year, end_year, cat_query)).as_dataframe()
df2.rename(columns={"country_name": "country", "count": "pubs"}, inplace=True)
df2 = df2[['name', 'country', 'pubs', 'rcr_avg', 'fcr_gavg', 'altmetric_median', ]]
df2['type'] = "research_organizations"
df2
#
# append second query to first
#
df = df.append(df2, ignore_index=True)
#
#
#
q3 = """search publications where open_access_categories.id="oa_all" and year in [{}:{}] and {} return research_orgs aggregate count, rcr_avg, fcr_gavg, altmetric_median limit 1000"""
df3 = dsl.query(q3.format(start_year, end_year, cat_query)).as_dataframe()
df3.rename(columns={"country_name": "country", "count": "pubs"}, inplace=True)
df3 = df3[['name', 'country', 'pubs', 'rcr_avg', 'fcr_gavg', 'altmetric_median', ]]
df3['type'] = "research_organizations_all_oa"
#df3
#
# append second query to first
#
df = df.append(df3, ignore_index=True)
# #

#
save_locally("DataTopTargetInstitutions", df)
#




###################
#
# 4. DATA TOP COUNTRIES 
#
###################


print("\n===\nData Top Countries\n===\n...extracting data..\n")
#
#
#
#
q1 = """search publications where year in [{}:{}] and {} return research_org_countries aggregate citations_total limit 1000"""
df = dsl.query(q1.format(start_year, end_year, cat_query)).as_dataframe()
df.rename(columns={"count": "pubs", "citations_total": "citations"}, inplace=True)
df = df[['name', 'pubs', 'citations', ]]
#df
#
#
# search grants now
#
q2 = """search grants where start_year in [{}:{}] and {} return research_org_countries aggregate funding limit 1000"""
df2 = dsl.query(q2.format(start_year, end_year, cat_query_grants), verbose=True).as_dataframe()
df2.rename(columns={"count": "grants"}, inplace=True)
#df2
if not df2.empty:
  df3 = pd.merge(df, df2[['name', 'grants', 'funding']], how='left', on="name")
else:
  df3 = df2
df3 = df3.fillna(0) # fill empty values
#
#
save_locally("DataTopCountries", df3)
#
#



###################
#
# 5. SAVING DATA
#
###################

print("\n===\nZipping up and downloading...\n")

import zipfile
from google.colab import files

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile(output_folder_name + '.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir(output_folder_name + '/', zipf)
zipf.close()


files.download(output_folder_name + '.zip')

print("\n===\nDone.")


*****
*****
Selection:
subject1: 0911 Maritime Engineering
subject2: None       
connector: or
years: 2015 - 2019
*****
*****


===
Data Competitors
===
...extracting data..



Returned Publisher: 363
Time: 0.5762450695037842s



..saved file 'DataCompetitor_0911_Maritime_Engineering_2015_2019.csv

===
Data Growth
===
...extracting data..



Returned Open_access_categories: 8
Time: 0.4623579978942871s


Returned Open_access_categories: 8
Time: 0.4586679935455322s


...


..saved file 'DataTopTargetInstitutions_0911_Maritime_Engineering_2015_2019.csv

===
Data Top Countries
===
...extracting data..



Returned Research_org_countries: 140
Time: 0.618520975112915s


Returned Research_org_countries: 52
Time: 0.4871039390563965s


WARNINGS [1]
Field 'FOR' is deprecated in favor of category_for. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details

..saved file 'DataTopCountries_0911_Maritime_Engineering_2015_2019.csv

===
Zipping up and downloading...



ModuleNotFoundError: No module named 'google'

# Downloading 

The exports should download automatically. 
However if there's an error with downloading the file, just run the cell below.

In [3]:
files.download(output_folder_name + '.zip')

NameError: name 'files' is not defined